In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from bincounts import *

path = '/home/zack/stl/scoot/populationsim-master/example_msa_survey/all_outputs'
path = './populationsim-master/example_msa_survey/all_outputs'

In [2]:
# # Fix Popsim runs with infeasible tracts (no synth persons file)
# synth_hh = pd.read_csv(f"{path}/50_99/synthetic_households.csv")
# p_data = pd.read_csv(f"{path}/../scoot_socio.csv", dtype=str)
# p_data.reset_index(inplace=True)

# # Clean up the file generated from QGIS spatial join
# geo = pd.read_csv('./data/TIGER/msa_puma_tract_join.csv', usecols=['jmsa_GEOID','jpuma_GEOI','GEOID']).dropna()
# geo['REGION'] = '1'
# geo = geo[['REGION','jmsa_GEOID','GEOID']].astype(int)
# geo.columns = ['REGION','MSA','TRACT']

# # Get rid of Micropolitan Statistical Areas
# msa_ids = pd.read_csv('./data/TIGER/msa_list.csv')
# geo = geo[geo['MSA'].isin(msa_ids['GEOID'])]
# full_msa_list = pd.unique(geo['MSA'])

# # Limit to certain MSAs if desired, there are 925 total MSA + MicroSA, 384 MSA
# msas_to_use = full_msa_list[50:99]
# geo = geo[geo['MSA'].isin(msas_to_use)]

# # Get the list of msas, and tracts that are being synthesized
# msa_list = list(pd.unique(geo['MSA']))
# msa_list = [int(i) for i in msa_list]
# tract_list = list(pd.unique(geo['TRACT']))
# tract_list = [int(i) for i in tract_list]

# # Fake household for each person
# h_data = p_data[['index','user']].copy()
# h_data['REGION'] = '1'
# h_data['WGTP'] = 1.0

# # Allocate all sample data to every MSA (seed geography)
# # Every person is alone in a single household (id=index)
# # Every person/household is repeated for every MSA
# p_data_list = []
# h_data_list = []
# for msa in msa_list:
#     p_data_new = p_data.copy()
#     p_data_new['MSA'] = msa
#     p_data_list.append(p_data_new)
#     h_data_new = h_data.copy()
#     h_data_new['MSA'] = msa
#     h_data_list.append(h_data_new)
# p_data = pd.concat(p_data_list)
# p_data.reset_index(inplace=True)
# p_data.drop(['level_0','index'], axis=1, inplace=True)
# p_data.reset_index(inplace=True)
# h_data = pd.concat(h_data_list)
# h_data.reset_index(inplace=True)
# h_data.drop(['level_0','index'], axis=1, inplace=True)
# h_data.reset_index(inplace=True)

In [3]:
# ordered_cols = ['MSA', 'TRACT', 'index', 'household_id', 'user', 'age', 'gender',
#        'race', 'hispanic', 'usborn', 'edu', 'student', 'work', 'zipcode',
#        'hhsize', 'child', 'hhincome', 'idincome', 'disable', 'veh',
#        'bike']
# merged_hh = synth_hh.merge(p_data, on='index', how='left').drop(columns=['MSA_y'])
# merged_hh['MSA'] = merged_hh['MSA_x']
# merged_hh = merged_hh[ordered_cols]
# merged_hh.to_csv(f"{path}/50_99/synthetic_persons.csv")

In [4]:
# Save sample of populations, while keeping total count
os.listdir(path)

percent_to_sample = .1
popsim_sample = []
tract_control_totals = []

for foldername in os.listdir(path):
    print(foldername)
    synthetic_p = pd.read_csv(f"{path}/{foldername}/synthetic_persons.csv")
    sample = synthetic_p.sample(int(percent_to_sample*len(synthetic_p)))
    popsim_sample.append(sample)
    
    controls = pd.read_csv(f"{path}/{foldername}/final_TRACT_control_data.csv")
    tract_control_totals.append(controls)

popsim_sample = pd.concat(popsim_sample)
tract_control_totals = pd.concat(tract_control_totals)
popsim_sample.to_csv(f"{path}/../combined_stats/popsim_sample.csv")
tract_control_totals.to_csv(f"{path}/../combined_stats/tract_control_totals.csv")

50_99
200_249
100_149
350_
300_349
0_49
150_199
250_299


### Control Variable Distributions

In [5]:
# Compare the counts of variables in the synthetic population with those in the ACS marginal tables
# Compare full region, as well as a random sample of 10 or so tracts visually
# Each MSA will need to be compared

In [6]:
popsim_sample = pd.read_csv(f"{path}/../combined_stats/popsim_sample.csv")
tract_control_totals = pd.read_csv(f"{path}/../combined_stats/tract_control_totals.csv")
# geos = pd.read_csv(f"{path}/../../../data/TIGER/msa_puma_tract_join.csv")
# labels = pd.read_csv(f"{path}/../configs/controls.csv")

In [7]:
random_tracts = tract_control_totals.sample(5)

tract_geoids = []
var_names = []
synthetic_counts = []
marginal_counts = []

for i, tract in random_tracts.iterrows():
    geoid = int(tract['GEO_ID'][9:])
    persons = popsim_sample.loc[popsim_sample['TRACT']==geoid,]
    for i, control in labels.iloc[1:,].iterrows():
        synth_count = sum(eval(control['expression']))
        marg_count = tract[control['control_field']]
        
        tract_geoids.append(geoid)
        var_names.append(control['control_field'])
        synthetic_counts.append(synth_count*10)
        marginal_counts.append(marg_count)

random_tract_comparisons = pd.DataFrame(zip(tract_geoids, var_names, marginal_counts, synthetic_counts), columns=['tract_geoid','var_name','marginal_count','synthetic_count'])

NameError: name 'labels' is not defined

In [ ]:
random_tract_comparisons

In [ ]:
popsim_sample

In [ ]:
# # Generate Barplot Distributions
# fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10,10))
# fig.tight_layout(h_pad=3, w_pad=6)

# var_list = ['Income','Age','Sex','Education']

# for i,var in enumerate(var_list):
#     # Handle household-level variables
#     # if var in ['NP','VEH']:
#     #     pums_counts = list(get_bin_counts(pums_sample_h[var],bin_list[i])/sum(get_bin_counts(pums_sample_h[var],bin_list[i])))
#     #     popsim_counts = list(get_bin_counts(popsim_sample_h[var],bin_list[i])/sum(get_bin_counts(popsim_sample_h[var],bin_list[i])))

#     # Handle tables where ACS tabulates NAs different than PUMS
#     if var =='SCHL':
#         over_24_pums_sample = pums_sample_p[pums_sample_p['AGEP']>=25]
#         pums_counts = list(get_bin_counts(over_24_pums_sample[var],bin_list[i])/sum(get_bin_counts(over_24_pums_sample[var],bin_list[i])))
#         over_24_popsim_sample = popsim_sample_p[popsim_sample_p['AGEP']>=25] # Change to this after fixing SCHL >24 input controls
#         over_24_popsim_sample = popsim_sample_p
#         popsim_counts = list(get_bin_counts(over_24_popsim_sample[var],bin_list[i])/sum(get_bin_counts(over_24_popsim_sample[var],bin_list[i])))
        
#     # Otherwise handle plain person-level variables
#     else:
#         pums_counts = list(get_bin_counts(pums_sample_p[var],bin_list[i])/sum(get_bin_counts(pums_sample_p[var],bin_list[i])))
#         popsim_counts = list(get_bin_counts(popsim_sample_p[var],bin_list[i])/sum(get_bin_counts(popsim_sample_p[var],bin_list[i])))

#     # Bring together bin counts to make barplot
#     sampled_counts = pd.DataFrame()
#     sampled_counts['density'] = popsim_counts + pums_counts
#     sampled_counts['Source'] = ['Popsim' for i in range(1,len(bin_list[i]))] + ['Pums' for i in range(1,len(bin_list[i]))]
#     sampled_counts['Group'] = bin_names_list[i]*2
#     sns.barplot(x="density", y="Group", hue="Source", data=sampled_counts, ax=axes.flatten()[i], orient='h')
#     axes.flatten()[i].set_title(var_names_list[i])
#     axes.flatten()[i].set_xlabel(None)
#     axes.flatten()[i].set_ylabel(None)

In [ ]:
# fig

In [ ]:
# fig.savefig('control_distributions.png', dpi=300)